# Time Lock Puzzles and Timed Release Crypto




### Adi Shamir, Ronald L. Rivest and David A. Wagner.


---
##### Submitted By, 
###### Ketan Kiran Bhujange - 181CO227
###### Rajath C Aralikatti - 181CO241
###### Sangeeth S V - 181CO246





The goal of *'timed release crypto'* is to encrypt a message in such a way that it cannot be decrypted by anyone, not even the sender, until the intended amount of time has passed. This involves finding a 'time-lock puzzle' of sorts which makes it so that the message cannot be decrypted until this puzzle has been solved. This makes choosing the 'puzzle' very important. The most noteworthy thing is that the puzzle should not be parallelizable, i.e, the number of computers working on the puzzle should not affect the time it takes to solve the puzzle. <br><br>
For example, consider a situation where the result of one operation is required by the next operation. While a certain degree of parallelizability is possible for the single operation, there is no feasible way to start the next operation until the current operation is finished. This is the fundamental principle used to develop the time lock puzzle here.

In [1]:
# Import necessary packages
import os
import sys
import math
import re
import time
import random

The first task in creating the time lock puzzle as described by Adi Shamir et al. is to generate large primes p and q and also compute n = p\*q along with its Euler's toitient function, phi(n) = (p-1) \* (q-1). 

In [2]:
# Fermat's primality test acts as a preliminary test of whether a number is prime or not.
    # a^(p-1) ≡ 1 mod p
    # Input: prime candidate p and security paramter s
    # Output: either p is a composite (always trues), or
            # p is a prime (with probability)
    # The security parameter is just the number of values of a which is checked. 


def fermat_primality_test(p, s=5):
    if p == 2:
        return True
    if not p & 1: # if p is even, number cant be a prime
        return False

    for i in range(s):
        a = random.randrange(2, p-2)
        x = pow(a, p-1, p) # a**(p-1) % p
        # The pow function in python efficiently computes the power of first two parameters modulo the third parameter.
        if x != 1:
            return False
    return True

# Testing
print(fermat_primality_test(53))
print(fermat_primality_test(51))

True
False


The Fermat's primality test returns prime with a probability. Therefore it is necessary to use a more deterministic test for primality checking. We will use Miller-Rabin's primality test.

In [3]:
def square_and_multiply(x, k, p=None):
    # Square and Multiply Algorithm
    # Parameters: positive integer x and integer exponent k,
    #             optional modulus p
    # Returns: x**k or x**k mod p when p is given

    b = bin(k).lstrip('0b')
    r = 1
    for i in b:
        r = r**2
        if i == '1':
            r = r * x
        if p:
            r %= p
    return r

def witness(a, r, p):
    # Returns: True, if there is an Euler witness that p is not prime.
    #          False, when p might be prime
    
    z = square_and_multiply(a, r, p)
    if z == 1:
        return False

    for i in range(u):
        z = square_and_multiply(a, 2**i * r, p)
        if z == p1:
            return False
    return True

def miller_rabin_primality_test(p, s=5):
    # 2 is the only even prime number
    if p == 2: 
        return True
    if not (p & 1):
        return False
    if not fermat_primality_test(p):
      return False
    # Here, we bring p-1 to 2^u * t form where t is odd.
    p1 = p - 1
    u = 0
    t = p1
    while t % 2 == 0:
        t >>= 1
        u += 1
    # at this stage p-1 = 2**u * t  holds
    assert p-1 == 2**u * t

    def witness(a):
        # Returns: True, if there is an Euler witness that p is not prime.
        #          False, when p might be prime
        
        z = square_and_multiply(a, t, p)
        if z == 1:
            return False

        for i in range(u):
            z = square_and_multiply(a, 2**i * t, p)
            if z == p1:
                return False
        return True
    
    # We are checking for a witness 's' number of times. s is the security parameter.
    for j in range(s):
        a = random.randrange(2, p-2)
        if witness(a):
            return False

    return True


# Testing
print(miller_rabin_primality_test(53))
print(miller_rabin_primality_test(51))

True
False


Now, we need to generate primes p and q and also compute phi(n).

In [4]:
def generate_primes(n=512, k=1):
    # Generates prime numbers with bitlength n.
    # Stops after the generation of k prime numbers.
    # Caution: The numbers tested for primality start at
    # a random place, but the tests are drawn with the integers
    # following from the random start.
    
    assert k > 0
    assert n > 0 and n < 4096

    # follows from the prime number theorem
    necessary_steps = math.floor( math.log(2**n) / 2 )
    
    # get n random bits as our first number to test for primality
    x = random.getrandbits(n)

    primes = []

    while k>0:
        if miller_rabin_primality_test(x, s=7):
            primes.append(x)
            k = k-1
        x = x+1

    return primes

# testing
print(generate_primes(n=512, k=1)[0])
print(generate_primes(n=512, k=1)[0])

11173788662624002577865892993025230120288901237640303861473786112667325931745475113409647115535535349388737678189666455975840379263190057558129149428405541
391353543213476929108265248133783892756457732662626055031057568473464821670324896023190645787462757374299634592568905557726871189525560149720985166774517


Now, the second step is to find the number of squarings per second.

In [5]:
count = 0
exp_b = random.getrandbits(1024)
exp_n = random.getrandbits(2048)

t_end = time.time() + 10
while time.time() < t_end:
    count += 1
    exp_b = exp_b ** 2 % exp_n

# count = count // 10
print(count//10)

69772


In [6]:
squarings_per_second = count//10

Now we need a random key for a conventional cryptosystem such as Fernet, which is an AES based assymetric encryption protocol

In [7]:
!pip install cryptography
from cryptography.fernet import Fernet

     |████████████████████████████████| 3.2MB 8.7MB/s 


The paper also talks about a faster way to compute b = a^(2^t) where 2^t is first computed as e and then b is computed as a^e.

In [8]:

def successive_squares(base: int, mod: int, length: int) -> [int]:
    table = [base % mod]
    prev = base % mod
    for n in range(1, length):
        squared = prev**2 % mod
        table.append(squared)
        prev = squared
    return table


def fast_exponentiation(n: int, g: int, x: int) -> int:
    # reverses binary string
    binary = bin(x)[2:][::-1]
    squares = successive_squares(g, n, len(binary))
    # keeps positive powers of two
    factors = [tup[1] for tup in zip(binary, squares) if tup[0] == '1']
    acc = 1
    for factor in factors:
        acc = acc * factor % n
    return acc

Now, we have everything we need for the encryption process

In [9]:
# The encryption process takes in three parameters
#   message: as a string of bytes
#   seconds: an integer, a measure of how much time has to be delayed
# 
# It returns 
#   n: p*q, p and q are hidden
#   a: safe pseudorandom number, can be 2 also.
#   t: seconds*squarings_per_second
#   encrypted_key: the encrypted key obtained after encryption of the key 
#                   generated by Fernet
#   encrypted_message: the message after encryption
def encrypt(message: bytes, seconds: int):
    if not seconds or not squarings_per_second:
        raise AssertionError

    # 1.  Generation of two large primes p and q and computing n and phi(n)
    p = generate_primes(n=1024, k=1)[0]
    q = generate_primes(n=1024, k=1)[0]
    n = p * q
    phi_n = (p - 1) * (q - 1)

    # 2.  Computing t = time * squarings_per_second
    t = seconds * squarings_per_second
    
    # 3.  Generation of a random key for a conventional cryptosystem like fernet.
    #     Fernet is an asymmetric encryption protocol using AES
    key = Fernet.generate_key()
    key_int = int.from_bytes(key, sys.byteorder)
    cipher_suite = Fernet(key)
    # print(f'{key} \n {key_int} \n {cipher_suite}')

    # 4.  Encryption using the chosen cryptosystem
    encrypted_message = cipher_suite.encrypt(message)

    # 5.  Pick safe, pseudo-random a where 1 < a < n
    #     Alternatively, we could use a = 2
    a = int.from_bytes(os.urandom(32), sys.byteorder) % n + 1
    assert a!=1 and a!=n

    # 6.  Key Encryption
    e = pow(2, t, phi_n)
    # b = fast_exponentiation(n, a, e)
    b = pow(a, e, n)
    encrypted_key = (key_int % n + b) % n
    
    
    return n, a, t, encrypted_key, encrypted_message

In [10]:
# Testing the encryption
print(encrypt(message='Vote'.encode(), seconds=10))

(2066073643581088238096268066130733059810696617763110420757814734683515031949490265917515572525156325969688456402746629417058443844800520382925166208989995591000572182028366031216513131405796807635876355810326024756103983923062537014761077510016500181921624718664404073378269672602113836068381864607061586876794267981309705992285410694163565538701449428354088910335842693294774978035947419570250483271936858604721288843898043046755580032574680807756506893887642115072522989536916780739360631842996661847148318435526423242379349769778081474272702801146391842451491172006543548504818812374585882577838145692635591352313, 54174469325431246892946132370458396673065080252676372190836587729148523033542, 697720, 166512062518812705233741486184053760999222466694244754100143332077052210698842016527482354987076214943309257705622304414377278451318839852543067077028582864824857600200250102648276541416161024313086734521327025615556290196073095129984537752090571083375778798808306853494205634283371059819124495

The decryption process is where the time lock puzzle plays its role. Here, we have assumed successive squaring to be the non-parallelizable operation. That is, the result of one operation is required for the next operation to begin.

In [11]:
# The decrypt method takes in 5 parameters:
#     n:  p*q
#     a:  safe pseudorandom number
#     t:  seconds * squarings_per_second
#     enc_key:  the encryption key generated by the encrypt method
#     enc_message: the encrypted message obtained from the encrypt method
#  It returns the decrypted message after enough time has passed
def decrypt(n: int, a: int, t: int, enc_key: int, enc_message: int) -> bytes:
    # Successive squaring to find b
    # We assume this cannot be parallelized
    b = a % n
    for i in range(t):
        b = b**2 % n
    dec_key = (enc_key - b) % n

    # Retrieve key, decrypt message
    key_bytes = int.to_bytes(dec_key, length=64, byteorder=sys.byteorder)
    cipher_suite = Fernet(key_bytes)
    return cipher_suite.decrypt(enc_message)

Both decrypt and encrypt functions have been defined. All that's left is to run it and measure the time taken to complete the execution.

In [12]:
def run_time_lock_puzzle(message: str, seconds: int, repeats: int):
    n, a, t, encrypted_key, encrypted_message = encrypt(message.encode(), seconds)

    times_taken = []
    for i in range(repeats):
      operation_time = time.time()
      print('Decrypting')
      print(decrypt(n, a, t, encrypted_key, encrypted_message).decode())
      times_taken.append(time.time()-operation_time)
      print(times_taken[-1])

    print(f'Average time taken for decryption = {sum(times_taken)/len(times_taken)} seconds.') 

In [15]:
message = str(input('Enter Message : '))
seconds = int(input('Enter Time to Delay in seconds : '))

Enter Message : gdasdgdsgfdsads
Enter Time to Delay in seconds : 15


In [16]:
run_time_lock_puzzle(message=message, seconds=seconds, repeats = 1)

Decrypting
gdasdgdsgfdsads
14.268259763717651
Average time taken for decryption = 14.268259763717651 seconds.


In [19]:
os.urandom(32)

UnicodeDecodeError: ignored